In [1]:
import pandas as pd
import pyarrow.parquet as pq
from dask import dataframe as dd
import time
import dask.dataframe as dd
import requests
import os
import dask.array as da
import os
import requests
import shutil
import dask.dataframe as dd
import time


# Taxon IDs

In [6]:
# Step 1: Read the taxon file and extract desired taxon IDs
desired_taxon_ids = set()

with pq.ParquetFile('taxons_data.parquet') as parquet_file:
    num_row_groups = parquet_file.num_row_groups
    for i in range(num_row_groups):
        batch = parquet_file.read_row_group(i).to_pandas()
        print(batch)
        desired_taxon_ids.update(set(batch['taxon_id']))

         taxon_id                        name
14447       48059          Prosopis juliflora
17141       47911           Asclepias syriaca
18194       48537       Lysichiton americanus
29238       57278         Ailanthus altissima
33892       52714           Cortaderia jubata
39834       77310           Gunnera tinctoria
42074       76766           Ehrharta calycina
43405       75433       Andropogon virginicus
44074       79388           Triadica sebifera
53725       84813       Baccharis halimifolia
72928      116710       Microstegium vimineum
106450     130228    Heracleum mantegazzianum
133682     159852  Cardiospermum grandiflorum
143330     164566           Lespedeza cuneata
144557     164985          Lygodium japonicum
298082     321137               Hakea sericea
416825     430581           Cenchrus setaceus
475919     499936        Heracleum sosnowskyi
482302     516305          Heracleum persicum
823888     876278       Microstegium vimineum
823890     876279       Microstegi

In [4]:
desired_taxon_ids

{47911,
 48059,
 48537,
 52714,
 57278,
 75433,
 76766,
 77310,
 79388,
 84813,
 116710,
 130228,
 159852,
 164566,
 164985,
 321137,
 430581,
 499936,
 516305,
 876278,
 876279,
 1293182}

# Photos

In [5]:
# large file: dask

start = time.time()
dask_photo_df = dd.read_csv('./photos.csv/photos.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.017539262771606445 sec


In [6]:
dask_photo_df.head()

,photo_uuid\tphoto_id\tobservation_uuid\tobserver_id\textension\tlicense\twidth\theight\tposition
0,8d6b2534-d30a-47a8-bc1c-986a21817997\t21213\t7...
1,6e8112fd-f703-4052-94da-b7cfc03ff3d4\t21216\t7...
2,49141c2f-48b0-4671-9cee-fce6efd11822\t21215\t0...
3,71090faa-9110-4df7-bb8f-af2415fe2e72\t21214\t5...
4,92c703d0-20f1-4da9-af9c-2a6bdb6db53b\t21217\te...


In [7]:
dask_photo_df.columns

Index(['photo_uuid\tphoto_id\tobservation_uuid\tobserver_id\textension\tlicense\twidth\theight\tposition'], dtype='object')

In [8]:
# Split the single column into multiple columns using the separator '\t'
dask_photos_df_split = dask_photo_df[dask_photo_df.columns[0]].str.split('\t', n=8, expand=True)

# Define column names
column_names = ['photo_uuid', 'photo_id', 'observation_uuid', 'observer_id', 'extension', 'license', 'width', 'height', 'position']

# Rename columns
dask_photos_df_split.columns = column_names

# Define meta for the taxon_id column
meta = ('photo_id', 'float64')

# Convert taxon_id to numeric using apply with meta
dask_photos_df_split['photo_id'] = dask_photos_df_split['photo_id'].apply(pd.to_numeric, errors='coerce', meta=meta)

# Drop rows with NaN in taxon_id column
dask_photos_df_split = dask_photos_df_split.dropna(subset=['photo_id'])

In [9]:
dask_photos_df_split.head()

,photo_uuid,photo_id,observation_uuid,observer_id,extension,license,width,height,position
0,8d6b2534-d30a-47a8-bc1c-986a21817997,21213,7ae155fc-f49e-4e4f-91c5-51e31e805478,516,jpg,CC-BY-NC,584,389,0
1,6e8112fd-f703-4052-94da-b7cfc03ff3d4,21216,7ae155fc-f49e-4e4f-91c5-51e31e805478,516,jpg,CC-BY-NC,584,389,1
2,49141c2f-48b0-4671-9cee-fce6efd11822,21215,05baefa2-028c-4c6a-b459-37f7161ca374,516,jpg,CC-BY-NC,584,389,0
3,71090faa-9110-4df7-bb8f-af2415fe2e72,21214,5c63a4aa-9828-4f6f-b181-7e71b4af24fa,516,jpg,CC-BY-NC,584,389,0
4,92c703d0-20f1-4da9-af9c-2a6bdb6db53b,21217,e9bfdce3-556c-4506-890b-9e65d241a929,516,jpg,CC-BY-NC,584,389,0


# Observations

In [10]:
# Read the CSV file with Dask
start = time.time()
dask_observations_df = dd.read_csv('./observations.csv/observations.csv/observations.csv')
end = time.time()
print("Read csv with dask:", (end - start), "sec")

Read csv with dask: 0.0196378231048584 sec


In [11]:
dask_observations_df.columns

Index(['observation_uuid\tobserver_id\tlatitude\tlongitude\tpositional_accuracy\ttaxon_id\tquality_grade\tobserved_on'], dtype='object')

In [12]:
dask_observations_df.head()

,observation_uuid\tobserver_id\tlatitude\tlongitude\tpositional_accuracy\ttaxon_id\tquality_grade\tobserved_on
0,7ae155fc-f49e-4e4f-91c5-51e31e805478\t516\t32....
1,05baefa2-028c-4c6a-b459-37f7161ca374\t516\t33....
2,5c63a4aa-9828-4f6f-b181-7e71b4af24fa\t516\t33....
3,e9bfdce3-556c-4506-890b-9e65d241a929\t516\t33....
4,19ad8eca-938a-4031-ba75-edee2749b247\t516\t32....


In [13]:
# Split the single column into multiple columns using the separator '\t'
dask_observation_df_split = dask_observations_df[dask_observations_df.columns[0]].str.split('\t', n=7, expand=True)

# Define column names
column_names = ['observation_uuid', 'observer_id', 'latitude', 'longitude', 'positional_accuracy', 'taxon_id', 'quality_grade', 'observed_on']

# Rename columns
dask_observation_df_split.columns = column_names

# Define meta for the taxon_id column
meta = ('taxon_id', 'float64')

# Convert taxon_id to numeric using apply with meta
dask_observation_df_split['taxon_id'] = dask_observation_df_split['taxon_id'].apply(pd.to_numeric, errors='coerce', meta=meta)

# Drop rows with NaN in taxon_id column
dask_observation_df_split = dask_observation_df_split.dropna(subset=['taxon_id'])

In [14]:
dask_observation_df_split['taxon_id'] = dask_observation_df_split['taxon_id'].astype(int)

In [15]:
dask_observation_df_split.head()

,observation_uuid,observer_id,latitude,longitude,positional_accuracy,taxon_id,quality_grade,observed_on
0,7ae155fc-f49e-4e4f-91c5-51e31e805478,516,32.1899345177,-80.7584837079,,203485,research,2009-10-31
1,05baefa2-028c-4c6a-b459-37f7161ca374,516,33.4306088000,-111.0270616000,,153887,research,2009-08-15
2,5c63a4aa-9828-4f6f-b181-7e71b4af24fa,516,33.4047428829,-111.9373476505,,313499,research,2009-08-22
3,e9bfdce3-556c-4506-890b-9e65d241a929,516,33.4306088000,-111.0270616000,,790491,research,2009-08-15
4,19ad8eca-938a-4031-ba75-edee2749b247,516,32.2163160000,-80.7526080000,,48505,research,2007-10-31


# Download

### All the taxons we want

In [16]:
taxon_observations_df = dask_observation_df_split[dask_observation_df_split['taxon_id'].isin(desired_taxon_ids)]

In [17]:
taxon_observations_df.head(5)

,observation_uuid,observer_id,latitude,longitude,positional_accuracy,taxon_id,quality_grade,observed_on
1806,f56b4072-7a0d-40c1-b5fb-2f999cd47b44,343,47.9133110000,-122.4231050000,,48537,research,2011-04-10
2024,da3cde42-e9b6-4b31-8467-d8ed8b9c418e,1,37.2087478638,-122.3335647583,76,48537,research,2011-04-19
7291,c85e1378-4fcf-4e8d-8219-e567e4043c32,2179,43.8687553406,-73.1087875366,10,47911,research,2011-08-21
7721,00468f3e-92ad-4a7d-b3d9-0622f34566c9,2179,44.0239821705,-73.1925539733,,47911,research,2011-09-06
8281,7421f879-8d6e-4a3f-a3ea-a36730d93396,477,37.6891250610,-122.4411163330,50,52714,research,2011-09-17


### The images with the columns we want

In [18]:
# Select only the required columns from the photo DataFrame
filtered_photos_df = dask_photos_df_split[['photo_id', 'photo_uuid', 'observation_uuid', 'extension']]

In [19]:
filtered_photos_df.head(5)

,photo_id,photo_uuid,observation_uuid,extension
0,21213,8d6b2534-d30a-47a8-bc1c-986a21817997,7ae155fc-f49e-4e4f-91c5-51e31e805478,jpg
1,21216,6e8112fd-f703-4052-94da-b7cfc03ff3d4,7ae155fc-f49e-4e4f-91c5-51e31e805478,jpg
2,21215,49141c2f-48b0-4671-9cee-fce6efd11822,05baefa2-028c-4c6a-b459-37f7161ca374,jpg
3,21214,71090faa-9110-4df7-bb8f-af2415fe2e72,5c63a4aa-9828-4f6f-b181-7e71b4af24fa,jpg
4,21217,92c703d0-20f1-4da9-af9c-2a6bdb6db53b,e9bfdce3-556c-4506-890b-9e65d241a929,jpg


### The observations

In [20]:
# Select only the required columns from the observation DataFrame
filtered_observation_df = dask_observation_df_split[['observation_uuid', 'taxon_id']]

In [21]:
filtered_observation_df.head(5)

,observation_uuid,taxon_id
0,7ae155fc-f49e-4e4f-91c5-51e31e805478,203485
1,05baefa2-028c-4c6a-b459-37f7161ca374,153887
2,5c63a4aa-9828-4f6f-b181-7e71b4af24fa,313499
3,e9bfdce3-556c-4506-890b-9e65d241a929,790491
4,19ad8eca-938a-4031-ba75-edee2749b247,48505


### Group photos per observation

In [22]:
# sample 10% of your data
subset_filtered_photos_df = filtered_photos_df.sample(frac=0.1, random_state=42) 

# Group photos by observation_uuid and aggregate them on the subset
grouped_photos_df = subset_filtered_photos_df.groupby('observation_uuid').agg({
    'photo_id': 'list',
    'photo_uuid': 'list',
    'extension': 'list'
}).reset_index()


In [23]:
grouped_photos_df.head(5)

In [ ]:
# Merge the filtered observation DataFrame with the grouped photo DataFrame
merged_df = dd.merge(grouped_photos_df, filtered_observation_df, on='observation_uuid', how='inner')

In [ ]:
merged_df.head(5)

In [ ]:
# Select only the required columns from the merged DataFrame
result_df = merged_df[['photo_id', 'photo_uuid', 'extension', 'observation_uuid', 'taxon_id']]

In [ ]:
result_df.head(5)

In [ ]:
unique_taxon_ids = result_df['taxon_id'].unique().compute()

In [ ]:
first_taxon_id = unique_taxon_ids[0]  # Assuming you've already computed unique_taxon_ids

rows_with_first_taxon_id = result_df[result_df['taxon_id'] == first_taxon_id].compute()

In [ ]:
# Compute the result DataFrame: takes a very very long time to compute
# result_df = result_df.compute()

In [ ]:
# Save the result DataFrame to a CSV file: takes a very very long time to compute
# result_df.to_csv('result_df*.csv', single_file=False)

In [ ]:
# Group the data by 'taxon_id'
# grouped_data = result_df.groupby('taxon_id')

In [ ]:
n = 30

# Define a function to save the CSV file for each taxon
def save_taxon_csv(group):
    taxon_id = group.name
    folder_name = f"{taxon_id}"
    os.makedirs(folder_name, exist_ok=True)  # Create a folder for the taxon
    
    
    # Save the list of photo IDs to a CSV file
    group[['photo_id']].to_csv(os.path.join(folder_name,  f"{taxon_id}_photo_list.csv"), index=False)

# Apply the function to each group and compute the result
# grouped_data.apply(save_taxon_csv).compute()